In [ ]:
import pandas as pd
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

In [ ]:
dayObs = 20251026
table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
table = table.sort_index()


In [ ]:
table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
table = table.sort_index()
fwhms = []
for i in range(len(table)):
    if (i > 464) and (i <= 503):
        fwhm = float(table.iloc[i]['PSF FWHM'])
        fwhms.append(fwhm)

print(np.nanmedian(fwhms))


In [ ]:
table.columns

In [ ]:
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay

startDay = 20250601
endDay = 20250720
dayObs = startDay
mountDegs = []
while dayObs <= endDay:
    NPts = 0
    try:
        table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
        table = table.sort_index()
        for i in range(len(table)):
            if 'SV' in table.iloc[i]['Target']:
                mountDeg = float(table.iloc[i]['Mount elevation RMS'])
                mountDegs.append(mountDeg)
                NPts += 1
    except:
        dayObs = calcNextDay(dayObs)
        continue
    print(dayObs, NPts)
    dayObs = calcNextDay(dayObs)


In [ ]:
plt.figure(figsize=(10,8))
maxMountDeg = 0.1
fails = [x for x in mountDegs if x > maxMountDeg]
plt.title("Mount motion image degradation for SV images\nIncludes Az, El, and Rotator\nBased on Encoder values.", fontsize = 18)
plt.hist(mountDegs, bins=100, range=(0, maxMountDeg))
plt.xlabel("Mount motion image degradation(arcseconds)", fontsize = 18)
plt.text(0.02, 250, f"{len(mountDegs)} images", fontsize = 18)
plt.text(0.02, 225, f'{(len(fails) / len(mountDegs) * 100.0):.1f} % of images are > 0.1"', fontsize = 18)
plt.text(0.02, 200, f"Median = {np.nanmedian(mountDegs):.2f} arcsec", fontsize = 18)
filename = "/home/cslage/DATA/Mount_Degradation_SV_20250601-20250720.png"
plt.savefig(filename)

In [ ]:
dayObs = 20250720
forces_720 = []
table = pd.read_json(f'/project/rubintv/tma/sidecar_metadata/dayObs_{dayObs}.json').T
table = table.sort_index()
for i in range(len(table)):
    forces_720.append(float(table.iloc[i]['HP Max force']))


In [ ]:
plt.hist(forces_719, bins=150, range=(0,2000), alpha=0.5, label='2025-07-19')
plt.hist(forces_720, bins=150, range=(0,2000), alpha=0.5, label='2025-07-20')
plt.legend()
plt.title("M1M3 HP Max force comparison")
plt.xlabel("HP Max force (N)")
filename = "/home/cslage/DATA/HP_Max_Force_Comparison.png"
plt.savefig(filename)

In [ ]:
startDay = 20250501
endDay = 20250728

limit = 0.25

outfilename = f"/home/cslage/DATA/Bad_Mount_Motion_Images_Fixed_{startDay}-{endDay}.txt"
outfile = open(outfilename, 'w')
outfile.write("ExpId \t\t\t Total \t\t\t Az \t\t El \t\t Rot\n")
dayObs = startDay
while dayObs <= endDay:
    try:
        table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
        table = table.sort_index()
        #print(dayObs, len(table))
        for i in range(1, len(table)+1):
            #if table['Can see the sky?'][i]:
            expId = int(dayObs * 1.0E5 + i)
            mountDeg = float(table['Mount motion image degradation'][i])
            mountDegEl = float(table['Mount elevation RMS'][i])
            mountDegAz = float(table['Mount azimuth RMS'][i])
            mountDegRot = float(table['Mount rotator RMS'][i])
            if mountDeg > limit:
                outfile.write(f"{expId} \t {mountDeg:.3f} \t {mountDegAz:.3f} \t {mountDegEl:.3f} \t {mountDegRot:.3f} \n" )
        dayObs = calcNextDay(dayObs) 
    except:
        print(f"{dayObs} failed")
        dayObs = calcNextDay(dayObs)
        continue
    print(f" Done with {dayObs}")
outfile.close()


In [ ]:
dayObs = 20250723
table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
table = table.sort_index()
print(dayObs, len(table))


In [ ]:
startDay = 20250501
endDay = 20250915

limit = 0.70

outfilename = f"/home/cslage/DATA/Great_Images_{startDay}-{endDay}.txt"
outfile = open(outfilename, 'w')
outfile.write("ExpId \t\t\t PSF FWHM \n")
dayObs = startDay
while dayObs <= endDay:
    try:
        table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
        table = table.sort_index()
        for i in range(1, len(table)+1):
            expId = int(dayObs * 1.0E5 + i)
            try:
                fwhm = float(table['PSF FWHM'][i])
            except:
                continue
            if fwhm < limit:
                outfile.write(f"{expId} \t {fwhm:.2f} \n" )
        dayObs = calcNextDay(dayObs) 
    except:
        print(f"{dayObs} failed")
        dayObs = calcNextDay(dayObs)
        continue
    print(f" Done with {dayObs}")
outfile.close()


In [ ]:
startDay = 20250501
endDay = 20250915
images = {}
dayObs = startDay
while dayObs <= endDay:
    try:
        table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
        table = table.sort_index()
        for i in range(1, len(table)+1):
            expId = int(dayObs * 1.0E5 + i)
            try:
                fwhm = float(table['PSF FWHM'][i])
            except:
                continue
            images[expId] = fwhm
        dayObs = calcNextDay(dayObs) 
    except:
        print(f"{dayObs} failed")
        dayObs = calcNextDay(dayObs)
        continue
    print(f" Done with {dayObs}")


In [ ]:
months = [202505, 202506, 202507, 202508, 202509]
names = ['May', 'June', 'July', 'August', 'September']
limits = [0.7, 0.8, 0.9]
counts = np.zeros([5,3], dtype='int')
totals = np.zeros([5], dtype='int')
for expId in images.keys():
    year_month = int(expId / 1E7)
    for i, month in enumerate(months):
        if year_month == month:
            totals[i] += 1
            for j, limit in enumerate(limits):
                if images[expId] < limit:
                    counts[i, j] +=1    

for i, month in enumerate(months):
    print(f"Total of {totals[i]} images in {names[i]}.")
    for j, limit in enumerate(limits):
        print(f"In {names[i]} there were {counts[i, j]} images with FWHM better that {limits[j]} arcseconds")
    print()

In [ ]:
startDay = 20250715
endDay = 20250817

limit = 1.0
test_el = 55
tol = 0.5

outfilename = f"/home/cslage/DATA/Hex_Motions_{test_el}_{startDay}-{endDay}.txt"
outfile = open(outfilename, 'w')
outfile.write("ExpId \t\t\t PSF FWHM \n")
dayObs = startDay
while dayObs <= endDay:
    try:
        table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
        table = table.sort_index()
        for i in range(1, len(table)+1):
            expId = int(dayObs * 1.0E5 + i)
            try:
                fwhm = float(table['PSF FWHM'][i])
                el = float(table['Elevation'][i])
            except:
                continue
            if (fwhm < limit) and (el > test_el - tol) and (el < test_el + tol):
                outfile.write(f"{expId} \t {fwhm:.2f}  \t {el:.3f} \n" )
        dayObs = calcNextDay(dayObs) 
    except:
        print(f"{dayObs} failed")
        dayObs = calcNextDay(dayObs)
        continue
    print(f" Done with {dayObs}")
outfile.close()


In [ ]:
table.columns

In [ ]:
fwhm = table['_PSF FWHM']
for f in fwhm:
    if not np.isnan(float(f)):
        print(f)
#fwhm

In [ ]:
len(table)

In [ ]:
filename = "/home/cslage/DATA/failures_20250718.pkl"
with open(filename, 'rb') as f:
    rmss = pkl.load(f)


In [ ]:
startDay = 20250718
endDay = 20250718
dayObs = startDay
while dayObs <= endDay:
    try:
        table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
        table = table.sort_index()
        for i in range(1, len(table)+1):
            expId = int(dayObs * 1.0E5 + i)
            try:
                fwhm = float(table['PSF FWHM'][i])
            except:
                continue
            if fwhm < limit:
                outfile.write(f"{expId} \t {fwhm:.2f} \n" )
        dayObs = calcNextDay(dayObs) 
    except:
        print(f"{dayObs} failed")
        dayObs = calcNextDay(dayObs)
        continue
    print(f" Done with {dayObs}")
outfile.close()


In [ ]:
dayObs = 20250912
table = pd.read_json(f'/project/rubintv/LSSTCam/guiders/sidecar_metadata/dayObs_{dayObs}.json').T
table = table.sort_index()


In [ ]:
table.columns

In [ ]:
startDay = 20250825
endDay = 20250918
azs = []
els = []
rots = []
az_drifts = []
el_drifts = []
rot_drifts = []
dayObs = startDay
while dayObs <= endDay:
    try:
        table = pd.read_json(f'/project/rubintv/LSSTCam/guiders/sidecar_metadata/dayObs_{dayObs}.json').T
        table = table.sort_index()
        for i in range(1, len(table)+1):
            expId = int(dayObs * 1.0E5 + i)
            try:
                az = float(table['Azimuth'][i])
                el = float(table['Elevation'][i])
                rot = float(table['Sky angle'][i])
                az_drift = float(table['Az drift (arcsec total)'][i])
                el_drift = float(table['Alt drift (arcsec total)'][i])
                rot_drift = float(table['Rotator drift (arcsec total)'][i])
                data = np.array([az, el, rot, az_drift, el_drift, rot_drift])
                if np.isnan(data).any():
                    continue
                azs.append(az)
                els.append(el)
                rots.append(rot)
                az_drifts.append(az_drift)
                el_drifts.append(el_drift)
                rot_drifts.append(rot_drift)
            except:
                continue
        dayObs = calcNextDay(dayObs) 
    except:
        print(f"{dayObs} failed")
        dayObs = calcNextDay(dayObs)
        continue
    print(f" Done with {dayObs}")


In [ ]:
len(azs)

In [ ]:
plt.scatter(azs, rot_drifts)

In [ ]:
dayObs = 20251026
table = pd.read_json(f'/project/rubintv/LSSTCamAos/sidecar_metadata/shards/dayObs_{dayObs}.json').T
table = table.sort_index()


In [ ]:
table.columns